In [1]:
import pandas as pd
import numpy as np
from binance.client import Client
client = Client()

In [2]:
# pull cyptocurrency data from binance for a specific sybol and start date
def get_data(symbol, start_date):
    data = pd.DataFrame(client.get_historical_klines(symbol, Client.KLINE_INTERVAL_4HOUR, start_date))
    df = data.iloc[:, :6]
    df.columns = ['date', 'open', 'high', 'low', 'close', 'volume']
    df.set_index('date', inplace=True)
    df.index = pd.to_datetime(df.index, unit='ms')
    df = df.astype(float)
    return df

In [3]:
df = get_data('BTCUSDT', '2021-01-01')

In [4]:
df

,open,high,low,close,volume
date,,,,,
2021-01-01 00:00:00,28923.63,29470.00,28690.17,29278.40,11560.456553
2021-01-01 04:00:00,29278.41,29395.00,28806.54,29092.83,7308.910274
2021-01-01 08:00:00,29092.84,29402.57,28872.24,29313.49,8283.705319
2021-01-01 12:00:00,29313.49,29600.00,29030.14,29188.67,11794.949515
2021-01-01 16:00:00,29188.67,29360.00,28624.57,29029.04,9850.965345
...,...,...,...,...,...
2022-11-19 04:00:00,16628.70,16649.00,16553.53,16599.31,19554.823030
2022-11-19 08:00:00,16598.98,16683.01,16580.00,16676.26,16369.326980
2022-11-19 12:00:00,16676.26,16686.86,16634.01,16655.65,17152.876880


In [5]:
# SMA crossover strategy
def sma_crossover(df, short, long):
    df['short'] = df['close'].rolling(window=short).mean()
    df['long'] = df['close'].rolling(window=long).mean()
    df['signal'] = np.where(df['short'] > df['long'], True, False)
    df['position'] = df['signal'] & (df['signal'].shift(1) == False)
    return df

In [6]:
# Add SMA_50 and SMA_100 to the dataframe
df = sma_crossover(df, 50, 100)
df.dropna(inplace=True)

In [10]:
# Add shifted open price to the dataframe
df['open_shift'] = df['open'].shift(-1)
df

,open,high,low,close,volume,short,long,signal,position,open_shift
date,,,,,,,,,,
2021-01-17 12:00:00,35270.00,35973.89,34666.00,35728.47,14071.341158,36780.9842,35456.7182,True,True,35728.47
2021-01-17 16:00:00,35728.47,36166.47,35514.99,35633.91,9298.406284,36682.3470,35520.2733,True,False,35633.90
2021-01-17 20:00:00,35633.90,36852.50,35506.00,35828.61,11388.103808,36588.4162,35587.6311,True,False,35824.99
2021-01-18 00:00:00,35824.99,36208.00,34842.99,35106.25,12865.201223,36477.2230,35645.5587,True,False,35108.27
2021-01-18 04:00:00,35108.27,36278.68,34800.00,36104.61,9931.587621,36397.5508,35714.7181,True,False,36100.55
...,...,...,...,...,...,...,...,...,...,...
2022-11-19 04:00:00,16628.70,16649.00,16553.53,16599.31,19554.823030,16748.2270,18258.3491,False,False,16598.98
2022-11-19 08:00:00,16598.98,16683.01,16580.00,16676.26,16369.326980,16740.4698,18222.6049,False,False,16676.26
2022-11-19 12:00:00,16676.26,16686.86,16634.01,16655.65,17152.876880,16725.5012,18187.6430,False,False,16655.65


In [11]:
in_position = False

buydates, selldate =[],[]
buyprices, sellprices =[],[]

for index,row in df.iterrows():
    if not in_position and row.position == True:
        buyprice = row.open_shift
        buydates.append(index)
        buyprices.append(buyprice)
        in_position = True
    if in_position:
        if row.low < buyprice * 0.99:
            sellprice = buyprice * 0.99
            selldate.append(index)
            sellprices.append(sellprice)
            in_position = False
        elif row.high > buyprice * 1.01:
            sellprice = buyprice * 1.01
            selldate.append(index)
            sellprices.append(sellprice)
            in_position = False

In [25]:
profits = [(sell-buy)/buy -0.0015 for sell,buy in zip(sellprices,buyprices)]

In [26]:
# calculate the profit by adding 1 and getting the product
profit = np.prod(np.array(profits)+1)
profit


0.8845156471937962

In [27]:
# create a function that calculates the profit for a given stop loss and take profit
def calculate_profit(df, stop_loss, take_profit):
    in_position = False
    buydates, selldate =[],[]
    buyprices, sellprices =[],[]
    for index,row in df.iterrows():
        if not in_position and row.position == True:
            buyprice = row.open_shift
            buydates.append(index)
            buyprices.append(buyprice)
            in_position = True
        if in_position:
            if row.low < buyprice * (1-stop_loss):
                sellprice = buyprice * (1-stop_loss)
                selldate.append(index)
                sellprices.append(sellprice)
                in_position = False
            elif row.high > buyprice * (1+take_profit):
                sellprice = buyprice * (1+take_profit)
                selldate.append(index)
                sellprices.append(sellprice)
                in_position = False
    profits = [(sell-buy)/buy -0.0015 for sell,buy in zip(sellprices,buyprices)]
    profit = np.prod(np.array(profits)+1)
    return profit

In [28]:
calculate_profit(df, 0.01, 0.01)

0.8845156471937962

In [29]:
# create stop loss range with np.arange
stop_loss_range = np.arange(0.01, 0.05, 0.01)
# create take profit range with np.arange
take_profit_range = np.arange(0.01, 0.05, 0.01)

In [30]:
# loop through the stop loss range and take profit range
for stop_loss in stop_loss_range:
    for take_profit in take_profit_range:
        profit = calculate_profit(df, stop_loss, take_profit)
        print(f'Stop Loss: {stop_loss}, Take Profit: {take_profit}, Profit: {profit}')

Stop Loss: 0.01, Take Profit: 0.01, Profit: 0.8845156471937962
Stop Loss: 0.01, Take Profit: 0.02, Profit: 0.8839904747466502
Stop Loss: 0.01, Take Profit: 0.03, Profit: 0.8491128071913759
Stop Loss: 0.01, Take Profit: 0.04, Profit: 0.8657047512954892
Stop Loss: 0.02, Take Profit: 0.01, Profit: 0.9102449968775022
Stop Loss: 0.02, Take Profit: 0.02, Profit: 0.8729292712898215
Stop Loss: 0.02, Take Profit: 0.03, Profit: 0.8543547259039295
Stop Loss: 0.02, Take Profit: 0.04, Profit: 0.8531114763209455
Stop Loss: 0.03, Take Profit: 0.01, Profit: 1.0579499467564548
Stop Loss: 0.03, Take Profit: 0.02, Profit: 0.9824047051276237
Stop Loss: 0.03, Take Profit: 0.03, Profit: 0.8770698154751725
Stop Loss: 0.03, Take Profit: 0.04, Profit: 0.8923744610731513
Stop Loss: 0.04, Take Profit: 0.01, Profit: 1.0790122689057027
Stop Loss: 0.04, Take Profit: 0.02, Profit: 0.9607254316771896
Stop Loss: 0.04, Take Profit: 0.03, Profit: 0.8915966509120951
Stop Loss: 0.04, Take Profit: 0.04, Profit: 0.915331267

In [34]:
#split the dataset to avoid overfitting
train = df[:int(len(df)*0.7)]
# test dataset
test = df[int(len(df)*0.7):]
# run the backtest on the training set
for stop_loss in stop_loss_range:
    for take_profit in take_profit_range:
        profit = calculate_profit(train, stop_loss, take_profit)
        print(f'Stop Loss: {stop_loss}, Take Profit: {take_profit}, Profit: {profit}')

Stop Loss: 0.01, Take Profit: 0.01, Profit: 0.910850330700272
Stop Loss: 0.01, Take Profit: 0.02, Profit: 0.8925217825036589
Stop Loss: 0.01, Take Profit: 0.03, Profit: 0.840717533207397
Stop Loss: 0.01, Take Profit: 0.04, Profit: 0.840717533207397
Stop Loss: 0.02, Take Profit: 0.01, Profit: 0.919037072198146
Stop Loss: 0.02, Take Profit: 0.02, Profit: 0.8818880680191884
Stop Loss: 0.02, Take Profit: 0.03, Profit: 0.8381907195610891
Stop Loss: 0.02, Take Profit: 0.04, Profit: 0.8130247970442566
Stop Loss: 0.03, Take Profit: 0.01, Profit: 1.0055639742675009
Stop Loss: 0.03, Take Profit: 0.02, Profit: 0.9733007327994818
Stop Loss: 0.03, Take Profit: 0.03, Profit: 0.8356371336383791
Stop Loss: 0.03, Take Profit: 0.04, Profit: 0.8179408134851066
Stop Loss: 0.04, Take Profit: 0.01, Profit: 1.0255833640624847
Stop Loss: 0.04, Take Profit: 0.02, Profit: 0.971786629460574
Stop Loss: 0.04, Take Profit: 0.03, Profit: 0.8672953340221136
Stop Loss: 0.04, Take Profit: 0.04, Profit: 0.85658025674289

In [35]:
# run the backtest on the test set
for stop_loss in stop_loss_range:
    for take_profit in take_profit_range:
        profit = calculate_profit(test, stop_loss, take_profit)
        print(f'Stop Loss: {stop_loss}, Take Profit: {take_profit}, Profit: {profit}')

Stop Loss: 0.01, Take Profit: 0.01, Profit: 0.9710878037599993
Stop Loss: 0.01, Take Profit: 0.02, Profit: 0.9904413450469779
Stop Loss: 0.01, Take Profit: 0.03, Profit: 1.0099858438207543
Stop Loss: 0.01, Take Profit: 0.04, Profit: 1.0297213000813294
Stop Loss: 0.02, Take Profit: 0.01, Profit: 0.9904333833894046
Stop Loss: 0.02, Take Profit: 0.02, Profit: 0.9898413448891656
Stop Loss: 0.02, Take Profit: 0.03, Profit: 1.0192844014680864
Stop Loss: 0.02, Take Profit: 0.04, Profit: 1.0493056047274618
Stop Loss: 0.03, Take Profit: 0.01, Profit: 1.0520961110675378
Stop Loss: 0.03, Take Profit: 0.02, Profit: 1.0093537095178755
Stop Loss: 0.03, Take Profit: 0.03, Profit: 1.0495821453701975
Stop Loss: 0.03, Take Profit: 0.04, Profit: 1.0910012636133115
Stop Loss: 0.04, Take Profit: 0.01, Profit: 1.0520961110675378
Stop Loss: 0.04, Take Profit: 0.02, Profit: 0.988617668274028
Stop Loss: 0.04, Take Profit: 0.03, Profit: 1.028019655977259
Stop Loss: 0.04, Take Profit: 0.04, Profit: 1.06858786483